In [1]:
from huggingface_hub import notebook_login

notebook_login()
#notebook login is niet werkend

/Users/kusmir29/Documents/localDev/Python Projects/ComputerVision/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<function huggingface_hub._login.notebook_login(new_session: bool = True, write_permission: bool = False) -> None>

In [2]:
# duurt mega lang!
from datasets import load_dataset
food = load_dataset("food101", split="train[:5000]")
food = food.train_test_split(test_size=0.2)

Generating validation split: 100%|██████████| 25250/25250 [00:00<00:00, 29028.33 examples/s]


In [3]:
food["train"][0]

{'image': <PIL.Image.Image image mode=RGB size=512x384>, 'label': 79}

In [4]:
# tonen van de image

im = food["train"][0]["image"]
im.show()

In [7]:
# dictionary methods van de dataset class van de HF lib

labels = food["train"].features["label"].names
print(type(labels))

<class 'list'>


In [9]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# i is de iteratie van de loop, label is de naam uit de list

In [10]:
# manueel zoeken naar een index 
print(id2label[str(23)])

churros


In [12]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k" #openai/clip-vit-base-patch32
image_processor = AutoImageProcessor.from_pretrained(checkpoint, use_fast=True) #use fast is beschikbaar bij dit model

In [13]:
'''Apply some image transformations to the images to make the model more robust against overfitting. 
Here you’ll use torchvision’s transforms module, 
but you can also use any image library you like.'''

from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [14]:
"""
Then create a preprocessing function to apply the transforms and return the pixel_values - the inputs to the model - of the image:

"""
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [15]:
'''
To apply the preprocessing function over the entire dataset, 
use 🤗 Datasets with_transform method. 
The transforms are applied on the fly when you load an element of the dataset:
'''

food = food.with_transform(transforms)

In [16]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [17]:
'''
Including a metric during training is often helpful for evaluating your model’s performance. 
You can quickly load an evaluation method with the 🤗 Evaluate library. For this task, 
load the accuracy metric (see the 🤗 Evaluate quick tour to learn more about how to load and compute a metric):
'''

import evaluate

accuracy = evaluate.load("accuracy")

In [18]:
# Then create a function that passes your predictions and labels to compute to calculate the accuracy:

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [19]:
# trainer is een functie dat werkt met de transformers lib om modellen makkelijker te trainen.
# Dit kan via de trainer functie of in pytorch 

from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
# checkpoint is een AutoImageProcessor
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = TrainingArguments(
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

  5%|▌         | 10/186 [00:32<07:52,  2.68s/it]

{'loss': 4.575, 'grad_norm': 1.5091770887374878, 'learning_rate': 2.6315789473684212e-05, 'epoch': 0.16}


 11%|█         | 20/186 [00:58<07:13,  2.61s/it]

{'loss': 4.2878, 'grad_norm': 1.9326695203781128, 'learning_rate': 4.970059880239521e-05, 'epoch': 0.32}


 16%|█▌        | 30/186 [01:24<06:47,  2.61s/it]

...

                                                 
100%|██████████| 186/186 [09:04<00:00,  2.60s/it]

{'eval_loss': 1.6427812576293945, 'eval_accuracy': 0.901, 'eval_runtime': 16.1342, 'eval_samples_per_second': 61.98, 'eval_steps_per_second': 3.905, 'epoch': 2.98}


100%|██████████| 186/186 [09:05<00:00,  2.93s/it]

{'train_runtime': 545.5651, 'train_samples_per_second': 21.996, 'train_steps_per_second': 0.341, 'train_loss': 2.4460641901980162, 'epoch': 2.98}


TrainOutput(global_step=186, training_loss=2.4460641901980162, metrics={'train_runtime': 545.5651, 'train_samples_per_second': 21.996, 'train_steps_per_second': 0.341, 'total_flos': 9.232831524962304e+17, 'train_loss': 2.4460641901980162, 'epoch': 2.976})

In [24]:
ds = load_dataset("food101", split="validation[:10]")
image = ds["image"][0]
image.show()

In [30]:
from transformers import pipeline

classifier = pipeline("image-classification", model="my_awesome_food_model/checkpoint-186", device='mps')
classifier(image)

[{'label': 'beignets', 'score': 0.970130205154419},
 {'label': 'ramen', 'score': 0.6129050254821777},
 {'label': 'prime_rib', 'score': 0.5952610373497009},
 {'label': 'bruschetta', 'score': 0.5819064378738403},
 {'label': 'pork_chop', 'score': 0.5796873569488525}]